In [1]:
import pandas as pd
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig,TrainingArguments,Trainer
from torch.utils.data import DataLoader, Dataset, random_split
from tqdm import tqdm
data = pd.read_csv("formatted_csv.csv")

In [2]:
data.columns

Index(['Speaker1', 'Speaker2'], dtype='object')

In [4]:
data.dtypes

Speaker1    object
Speaker2    object
dtype: object

In [5]:
len(data)

52456

In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from torch.utils.data import DataLoader, Dataset
import pandas as pd

In [ ]:
# Define your dataset class
class ConversationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        input_text = self.data['Speaker1'][index]
        target_text = self.data['Speaker2'][index]

        inputs = self.tokenizer.encode(input_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        targets = self.tokenizer.encode(target_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')

        return {'input_ids': inputs.squeeze(), 'labels': targets.squeeze()}

# Load the BART tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')


In [ ]:
# Prepare your data
# Replace this with loading your actual dataset
data = pd.read_csv('formatted_csv.csv') # Load your dataset here
max_length = 80  # Define a suitable maximum length for your sequences
dataset = ConversationDataset(data, tokenizer, max_length)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


In [ ]:
# Define the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in dataloader:
        input_ids = batch['input_ids'].to(model.device)
        labels = batch['labels'].to(model.device)

        optimizer.zero_grad()

        outputs = model(input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss}")


In [ ]:
# Generating responses
def generate_response(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors='pt', max_length=max_length, truncation=True)
    output_ids = model.generate(input_ids)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response
